In [1]:
### REGRESSOR - train model - SCORE

In [2]:
# Todd McCullough
# 2020
import re
import numpy as np
import pandas as pd
import cpl_main as cpl
import random

In [3]:
results = pd.read_csv(f'datasets/soccer-nn-train-pred.csv')
results.head(2)

,game,team,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,r,s,pred
0,I1,Forge FC,3.40,2.48,4.43,2.65,3.43,4.34,3.49,2.34,4.22,2.71,3.68,4.25,3.71,1.72,2,1,2
1,I1,York9 FC,4.09,4.06,2.42,2.71,3.46,2.30,3.55,2.83,3.64,1.93,4.76,3.70,4.47,3.45,2,1,2


In [4]:
pred = results.pop('pred')
results.shape

(4042, 18)

In [5]:
results = results[results['p1'] != 0]

In [6]:
results.shape

(4036, 18)

In [7]:
cols = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9','p10', 'p11', 'p12', 'p13', 'p14']

In [8]:
results['sum'] = round(results[cols].sum(axis=1),2)

In [9]:
results['count'] = (results[results[cols]>0].count(axis=1))-11

In [10]:
results['result'] = round(results['sum'] / (results['count']+0.1),2)

In [11]:
results['test'] = round(results['sum']  / results['sum'].max() - 0.05,3)

In [12]:
results['test2'] = round(results['count']  / results['count'].max() - 0.05,3)

In [13]:
results = results.dropna()

In [14]:
results.head(2)

,game,team,p1,p2,p3,p4,p5,p6,p7,p8,...,p12,p13,p14,r,s,sum,count,result,test,test2
0,I1,Forge FC,3.40,2.48,4.43,2.65,3.43,4.34,3.49,2.34,...,4.25,3.71,1.72,2,1,46.85,3,15.11,0.868,0.95
1,I1,York9 FC,4.09,4.06,2.42,2.71,3.46,2.30,3.55,2.83,...,3.70,4.47,3.45,2,1,47.37,3,15.28,0.878,0.95


In [15]:
cols2=[]
for col in cols:
    string = re.sub('p','n',col)
    cols2.append(string)
    results[string] = results[col].apply(lambda x: int(x))

In [16]:
results['sum'] = results['sum'] - results[cols2].sum(axis=1)

In [17]:
results['f'] = (results[results[cols2]==4].count(axis=1))
results['m'] = (results[results[cols2]==3].count(axis=1))
results['d'] = (results[results[cols2]==2].count(axis=1))
results['g'] = (results[results[cols2]==1].count(axis=1))

In [18]:
results.f.unique()

array([4, 2, 3, 1, 5, 6, 7, 0])

In [19]:
results.m.unique()

array([ 5,  4,  6,  7,  8,  3,  2,  9, 10,  1,  0])

In [20]:
results.d.unique()

array([ 4,  5,  3,  6,  7,  2, 10,  8,  1,  9])

In [21]:
results.g.unique()

array([1, 2, 0])

In [22]:
results.head(2)

,game,team,p1,p2,p3,p4,p5,p6,p7,p8,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,I1,Forge FC,3.40,2.48,4.43,2.65,3.43,4.34,3.49,2.34,...,4,2,3,4,3,1,4,5,4,1
1,I1,York9 FC,4.09,4.06,2.42,2.71,3.46,2.30,3.55,2.83,...,3,1,4,3,4,3,4,5,4,1


In [23]:
db = results.copy()

In [24]:
db.shape

(4036, 41)

In [25]:
db.pop('game')
db.pop('team')
#y = db.pop('s')
db.pop('r')
X = db

In [26]:
X.head(2)

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,3.40,2.48,4.43,2.65,3.43,4.34,3.49,2.34,4.22,2.71,...,4,2,3,4,3,1,4,5,4,1
1,4.09,4.06,2.42,2.71,3.46,2.30,3.55,2.83,3.64,1.93,...,3,1,4,3,4,3,4,5,4,1


In [27]:
from math import sqrt

#importing libraries from sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn import metrics
# import algorithm modules
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [28]:
high = X[X['s'] > 2]
high.head(2)

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
12,3.40,4.39,2.48,2.71,3.36,2.65,3.43,4.34,3.49,4.22,...,3,4,2,2,4,1,4,4,5,1
43,2.83,4.70,3.46,3.45,2.38,4.39,3.53,3.68,1.79,2.33,...,1,2,2,4,4,3,4,5,4,1


In [29]:
h_y = high.pop('s')

In [30]:
low = X[X['s'] <= 2]
low.head(2)

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,3.40,2.48,4.43,2.65,3.43,4.34,3.49,2.34,4.22,2.71,...,4,2,3,4,3,1,4,5,4,1
1,4.09,4.06,2.42,2.71,3.46,2.30,3.55,2.83,3.64,1.93,...,3,1,4,3,4,3,4,5,4,1


In [31]:
l_y = low.pop('s')

In [32]:
print(high.shape,low.shape)

(754, 37) (3282, 37)


In [33]:
h_X_train, h_X_test, h_y_train, h_y_test = train_test_split(high, h_y, test_size=0.2, random_state=7)
seed = 7

In [34]:
l_X_train, l_X_test, l_y_train, l_y_test = train_test_split(low, l_y, test_size=0.2, random_state=7)
seed = 7

In [35]:
l_X_train = pd.concat([l_X_train,h_X_train])

In [36]:
l_X_test = pd.concat([l_X_test,h_X_test])

In [37]:
l_X_train.shape[0]

3228

In [38]:
l_X_train.shape[0] + l_X_test.shape[0]

4036

In [39]:
l_y_train = pd.concat([l_y_train,h_y_train])

In [40]:
l_y_test = pd.concat([l_y_test,h_y_test])

In [41]:
l_y_train.shape[0] + l_y_test.shape[0]

4036

In [42]:
l_y_test.shape[0] + l_X_test.shape[0]

1616

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
seed = 7

In [43]:
X_train, X_test, y_train, y_test = l_X_train, l_X_test, l_y_train, l_y_test

In [44]:
db.shape

(4036, 38)

In [45]:
X_train.shape

(3228, 37)

In [46]:
X_test.shape

(808, 37)

In [47]:
X_train.shape[0] + X_test.shape[0]

4036

In [48]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

In [49]:
kfold = KFold(n_splits=10, random_state=7,shuffle=True)

In [50]:
a=[]

In [51]:
import time
def train_rf(n_est, depth):
    rf = RandomForestRegressor(n_estimators=n_est,
                                max_depth=depth,
                                n_jobs=-1)
    start = time.time()
    rf_model = rf.fit(X_train,y_train)
    end = time.time()
    fit_time = round((end - start),3)
    start = time.time()
    y_pred = rf_model.predict(X_test)
    end = time.time()
    pred_time = round((end - start),3)
    #precision, recall, fscore, support = score(y_test, y_pred,average='macro') # pass the y labels to the score, the predicted, positive label; what we are predicting
    r2 = cross_val_score(rf,X_test,y_test,cv=kfold,scoring='r2')
    score = round(rf.score(X_train, y_train),3)
    a.append([score,fit_time,pred_time,n_est,depth,round(r2.mean(),3),round(r2.std(),3)])
    #print(f'time: {fit_time} pred: {pred_time} EST: {n_est} / depth: {depth} / r^2: {round(r2.mean(),3),round(r2.std(),3)} score: {score}')

In [52]:
for n_est in [10,50,100,200]:
    for depth in [10,20,80,None]:
        train_rf(n_est,depth)

In [53]:
final_rf = pd.DataFrame(a,columns=['score','fit time','pred_time','estimators','depth','mean','std'])
final_rf.sort_values(by=['score'],ascending=False)

,score,fit time,pred_time,estimators,depth,mean,std
14,0.862,2.632,0.112,200,80.0,-0.032,0.049
15,0.862,2.534,0.108,200,NaN,-0.032,0.054
11,0.860,1.375,0.115,100,NaN,-0.047,0.040
10,0.859,1.374,0.108,100,80.0,-0.038,0.042
7,0.855,0.639,0.109,50,NaN,-0.049,0.067
6,0.854,0.640,0.109,50,80.0,-0.039,0.062
9,0.844,1.174,0.109,100,20.0,-0.040,0.061
13,0.844,3.163,0.116,200,20.0,-0.033,0.045
5,0.834,0.538,0.106,50,20.0,-0.029,0.062
2,0.813,0.216,0.112,10,80.0,-0.076,0.078


In [54]:
b=[]

In [55]:
def train_dt(feat, depth):
    dt = DecisionTreeRegressor(criterion='mse',
                               splitter='random',
                               max_features=feat,
                               max_depth=depth)
    start = time.time()
    dt_model = dt.fit(X_train,y_train)
    end = time.time()
    fit_time = round((end - start),3)
    start = time.time()
    y_pred = dt_model.predict(X_test)
    end = time.time()
    pred_time = round((end - start),3)
    
    #precision, recall, fscore, support = score(y_test, y_pred,average='macro') # pass the y labels to the score, the predicted, positive label; what we are predicting
    r2 = cross_val_score(dt,X_test,y_test,cv=kfold,scoring='r2')
    score = round(dt.score(X_train, y_train),3)
    b.append([score,fit_time,pred_time,feat,depth,round(r2.mean(),3),round(r2.std(),3)])
    #print(f'time: {fit_time} pred: {pred_time} EST: {feat} / depth: {depth} / r^2: {round(r2.mean(),3),round(r2.std(),3)} score: {score}')

In [56]:
for feat in [5,10,X_train.shape[1],'auto','sqrt','log2',None]:
    for depth in [10,20,80,None]:
        train_dt(feat,depth)

In [57]:
final_dt = pd.DataFrame(b,columns=['score','fit time','pred_time','features','depth','mean','std'])
final_dt[final_dt['score'] < 0.98].sort_values(by=['score'],ascending=False)

,score,fit time,pred_time,features,depth,mean,std
25,0.912,0.021,0.002,None,20.0,-1.151,0.332
13,0.867,0.023,0.003,auto,20.0,-1.014,0.348
5,0.824,0.010,0.003,10,20.0,-0.964,0.217
17,0.805,0.008,0.003,sqrt,20.0,-0.798,0.390
9,0.795,0.022,0.003,37,20.0,-1.006,0.379
1,0.768,0.007,0.002,5,20.0,-0.938,0.321
21,0.620,0.007,0.002,log2,20.0,-0.979,0.234
8,0.336,0.013,0.003,37,10.0,-0.462,0.159
24,0.316,0.012,0.002,None,10.0,-0.510,0.230
12,0.266,0.013,0.003,auto,10.0,-0.504,0.297


In [58]:
def show_model_scores(model,string):
    r2 = cross_val_score(model,X_test,y_test,cv=kfold,scoring='r2')
    print(f'{string} r2: {round(r2.mean(),3),round(r2.std(),3)}')

#DecisionTreeRegressor
def decisionTree():
    model = DecisionTreeRegressor(criterion='mse',
                                  splitter='random',
                                  max_features=None,
                                  max_depth=20)
    model.fit(X_train, y_train)
    return model
dt = decisionTree()

show_model_scores(dt,'Decision Tree Regression Model')

Decision Tree Regression Model r2: (-1.095, 0.367)


In [59]:
#Random Forest Regression
def forestRegression():
    model = RandomForestRegressor(n_estimators=100,
                                  max_depth=None,
                                  criterion='mse',
                                  n_jobs=-1)
    model.fit(X_train, y_train)
    return model

rf = forestRegression()

show_model_scores(rf,'Random Forest Regression Model')

Random Forest Regression Model r2: (-0.045, 0.045)


In [60]:
from sklearn.ensemble import VotingRegressor
vr = VotingRegressor(estimators=[('dt1', dt),('dt2', dt),('dt3', dt)])
vr = vr.fit(X_train, y_train)

show_model_scores(vr,'Voting Regressor Model')

Voting Regressor Model r2: (-0.355, 0.22)


In [61]:
def check(a,b):
    #print('pred :',a,'actual :',b)
    if a == b:
        result = 1
    else:
        result = 0
    return result

In [62]:
print('Decission Tree',round(dt.score(X_train, y_train),2))
print('Random Forest',round(rf.score(X_train, y_train),2))
print('Voting Regressor DT',round(vr.score(X_train, y_train),2))

Decission Tree 0.88
Random Forest 0.86
Voting Regressor DT 0.88


In [63]:
def cycle_pred_test(result,num):
    score = check(result,y_test.loc[num])
    return score

def model_pred_test(model):
    pred = []
    numbers = X_test.index
    random_nums = random.choices(numbers, k=50)
    for i in random_nums:
        p = X_test.loc[i].tolist()
        result = model.predict([p]).flatten().round()
        prediction = cycle_pred_test(result,i)
        pred.append(prediction)
    dz = pd.DataFrame(pred)
    #df = pd.DataFrame(prob)
    c = str(float(dz.sum().values / 50))
    return c

In [64]:
test_results = pd.DataFrame(index=range(10),columns=['vr','dt','rf'])
test_results = test_results.fillna(0.0)

In [65]:
for i in range(10):
    v_s = model_pred_test(vr)
    test_results.at[i,'vr'] = v_s
    d_s = model_pred_test(dt)
    test_results.at[i,'dt'] = d_s
    r_s = model_pred_test(rf)
    test_results.at[i,'rf'] = r_s

In [66]:
test_results.describe()

,vr,dt,rf
count,10.000000,10.000000,10.00000
mean,0.230000,0.302000,0.25400
std,0.089567,0.061427,0.03134
min,0.100000,0.220000,0.20000
25%,0.180000,0.265000,0.24000
50%,0.210000,0.280000,0.25000
75%,0.315000,0.340000,0.26000
max,0.340000,0.420000,0.32000


In [67]:
def cycle_prob_test(num,model):
    p = X_test.iloc[num].tolist()
    e = model.predict([p]).flatten()
    e = e[0]
    return int(round(e))

def model_pred_test(model):
    b = []
    prob = []
    random_nums = np.random.randint(low=1, high=58, size=(20))
    for i in random_nums:
        prob.append(cycle_prob_test(i,model))
    df = pd.DataFrame(prob)
    df = df.values
    print('scores :\n',df)

In [68]:
model_pred_test(dt)

scores :
 [[0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [3]
 [0]
 [1]
 [0]
 [1]
 [3]
 [0]
 [1]
 [0]
 [2]
 [2]
 [4]
 [2]
 [1]]


In [69]:
model_pred_test(vr)

scores :
 [[0]
 [1]
 [2]
 [0]
 [1]
 [2]
 [2]
 [3]
 [0]
 [1]
 [2]
 [0]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]]


In [70]:
model_pred_test(rf)

scores :
 [[1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]]


In [71]:
import pickle
filename = 'models/cpl_score_regressor-08-19-rf.sav'
pickle.dump(rf, open(filename, 'wb'))

In [72]:
import pickle
filename = 'models/cpl_score_regressor-08-19-vr.sav'
pickle.dump(vr, open(filename, 'wb'))

In [71]:
import pickle
filename = 'models/cpl_score_regressor-08-19-dt.sav'
pickle.dump(dt, open(filename, 'wb'))